In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np
import shutil

# Define dataset path
dataset_path = "COVID-19_Radiography_Dataset"

# Image transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),                    # Converts to tensor (C, H, W)
    transforms.Normalize([0.5], [0.5])        # Simple normalization
])

# Load dataset
dataset = ImageFolder(root=dataset_path, transform=transform)

# Stratified split
targets = dataset.targets
train_idx, val_idx = train_test_split(
    list(range(len(targets))),
    test_size=0.3,
    stratify=targets,
    random_state=42
)

# Subset the dataset
from torch.utils.data import Subset
train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(f"Train set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Classes: {dataset.classes}")


Train set size: 29631
Validation set size: 12699
Classes: ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']


In [2]:
import torch
import torch.nn as nn
import torchvision.models as models

def get_model(name, width_mult=1.0, num_classes=4):
    """
    Loads the selected pretrained model and replaces the classifier
    with: 1024 → 512 → num_classes
    """
    if name == "mobilenet":
        model = models.mobilenet_v2(pretrained=True, width_mult=width_mult)

    elif name == "efficientnet_b0":
        model = models.efficientnet_b0(pretrained=True)
        
    elif name == "shufflenetv2":
        model = models.shufflenet_v2_x1_0(pretrained=True)
        if width_mult == 0.5:
            model = models.shufflenet_v2_x0_5(pretrained=True)
        elif width_mult == 1.5:
            model = models.shufflenet_v2_x1_5(pretrained=True)
        elif width_mult == 2.0:
            model = models.shufflenet_v2_x2_0(pretrained=True)

    elif name == "mnasnet":
        if width_mult == 0.5:
            model = models.mnasnet0_5(pretrained=True)
        elif width_mult == 1.0:
            model = models.mnasnet1_0(pretrained=True)
        else:
            raise ValueError("Only mnasnet0_5 and mnasnet1_0 supported in torchvision.")

    else:
        raise ValueError("Unsupported architecture")

    # Freeze all layers
    for param in model.parameters():
        param.requires_grad = False

    # Replace classifier
    in_features = None
    if name in ["mobilenet", "efficientnet_b0"]:
        in_features = model.classifier[1].in_features
    elif name == "shufflenetv2":
        in_features = model.fc.in_features
    elif name == "mnasnet":
        in_features = model.classifier.in_features

    classifier = nn.Sequential(
        nn.Linear(in_features, 1024),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )

    # Attach classifier
    if name in ["mobilenet", "efficientnet_b0", "mnasnet"]:
        model.classifier = classifier
    elif name == "shufflenetv2":
        model.fc = classifier

    return model


In [4]:
# Correct way: MobileNetV2 x1.0 with pretrained weights
from torchvision.models import mobilenet_v2

model = mobilenet_v2(pretrained=True)  # width_mult = 1.0 by default

# Freeze features
for param in model.features.parameters():
    param.requires_grad = False

# Replace classifier
num_classes = len(dataset.classes)
model.classifier = nn.Sequential(
    nn.Linear(model.last_channel, 1024),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, num_classes)
)
